In [6]:
import os
import requests
import zipfile
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# --- Paths ---
cwd = os.getcwd()
geo_dir = os.path.join(cwd, "geo")
os.makedirs(geo_dir, exist_ok=True)

data_file = os.path.join(os.path.dirname(cwd), "data", "fire_archive_M-C61_626683.csv.xz")

# --- URLs and files for 2024 TIGER/Line shapefiles ---
base_url = "https://www2.census.gov/geo/tiger/TIGER2024"

shapefiles = {
    "states": f"{base_url}/STATE/tl_2024_us_state.zip",
    "counties": f"{base_url}/COUNTY/tl_2024_us_county.zip",
}

# List of valid FIPS codes for states (1-56 excluding invalid ones)
valid_fips = [f"{i:02d}" for i in range(1, 79) if i not in [3, 7, 14, 43, 52, 57, 58, 59, 61, 62, 63, 64, 65, 67, 68, 70, 71, 73, 74, 75, 76, 77]]

# --- Download and extract function ---
def download_and_extract(url, extract_to):
    filename = os.path.basename(url)
    dest_path = os.path.join(extract_to, filename)

    # Download zip if not exists
    if not os.path.exists(dest_path):
        print(f"Downloading {filename} ...")
        try:
            r = requests.get(url, verify=False)
            r.raise_for_status()
            with open(dest_path, "wb") as f:
                f.write(r.content)
            print(f"Downloaded {filename}")
        except Exception as e:
            print(f"Failed downloading {filename}: {e}")
            return False
    else:
        print(f"{filename} already downloaded.")

    # Extract shapefile components if .shp does not exist yet
    shp_file = filename.replace(".zip", ".shp")
    shp_path = os.path.join(extract_to, shp_file)
    if not os.path.exists(shp_path):
        print(f"Extracting {filename} ...")
        with zipfile.ZipFile(dest_path) as z:
            for file in z.namelist():
                if file.endswith((".shp", ".shx", ".dbf", ".prj")):
                    z.extract(file, extract_to)
        print(f"Extracted shapefiles from {filename}")
    else:
        print(f"{shp_file} already extracted.")
    return True

# --- Download states and counties ---
for name, url in shapefiles.items():
    download_and_extract(url, geo_dir)

# --- Download places shapefiles ---
for fips in valid_fips:
    url = f"{base_url}/PLACE/tl_2024_{fips}_place.zip"
    download_and_extract(url, geo_dir)

# --- Load fire data ---
print("Loading fire data...")
data = pd.read_csv(data_file)
data['acq_date']=pd.to_datetime(data['acq_date'])
# Create datetime
data['acq_datetime'] = (
    data['acq_date'] +
    pd.to_timedelta(data['acq_time'] // 100, unit='h') +
    pd.to_timedelta(data['acq_time'] % 100, unit='m')
)
data['acq_datetime'] = data['acq_datetime'].dt.normalize()

data.drop(['acq_time', 'instrument'], axis=1, inplace=True)
data['confidence_binned'] = pd.cut(data['confidence'], bins=[-1, 30, 80, 101], labels=['l', 'n', 'h'])

# Create GeoDataFrame
geometry = [Point(xy) for xy in zip(data['longitude'], data['latitude'])]
fire_gdf = gpd.GeoDataFrame(data, geometry=geometry, crs="EPSG:4326")

# --- Load shapefiles ---

print("Loading states...")
states_path = os.path.join(geo_dir, "tl_2024_us_state.shp")
states_gdf = gpd.read_file(states_path)[["NAME", "geometry"]]
states_gdf = states_gdf.rename(columns={"NAME": "state_name"}).to_crs(epsg=4326)

print("Loading counties...")
counties_path = os.path.join(geo_dir, "tl_2024_us_county.shp")
counties_gdf = gpd.read_file(counties_path)[["NAME", "geometry"]]
counties_gdf = counties_gdf.rename(columns={"NAME": "county_name"}).to_crs(epsg=4326)

print("Loading places...")
place_gdfs = []
for file in os.listdir(geo_dir):
    if file.startswith("tl_2024_") and file.endswith("_place.shp"):
        gdf = gpd.read_file(os.path.join(geo_dir, file))[["NAME", "geometry"]]
        gdf = gdf.rename(columns={"NAME": "place_name"})
        place_gdfs.append(gdf)

places_gdf = pd.concat(place_gdfs, ignore_index=True)
places_gdf = gpd.GeoDataFrame(places_gdf, crs="EPSG:4269").to_crs(epsg=4326)

# --- Spatial joins ---

print("Mapping fire points to states...")
fire_gdf = gpd.sjoin(fire_gdf, states_gdf, how="left", predicate="within")
fire_gdf = fire_gdf.drop(columns=['index_right'])

print("Mapping fire points to counties...")
fire_gdf = gpd.sjoin(fire_gdf, counties_gdf, how="left", predicate="within")
fire_gdf = fire_gdf.drop(columns=['index_right'])

print("Mapping fire points to places...")
fire_gdf = gpd.sjoin(fire_gdf, places_gdf, how="left", predicate="within")
fire_gdf = fire_gdf.drop(columns=['index_right'])

data=fire_gdf
# --- Drop unnecessary columns ---
for col in ['STATEFP', 'COUNTYFP', 'PLACEFP']:
    if col in fire_gdf.columns:
        fire_gdf = fire_gdf.drop(columns=col)

import numpy as np
from sklearn.cluster import DBSCAN

# Earth radius in km
kms_per_radian = 6371.0088
radius_km = 0.75  # adjust for clustering range
eps = radius_km / kms_per_radian

results = []

for date, group in data.groupby('acq_date'):
    coords_rad = np.radians(group[['latitude', 'longitude']].values)

    db = DBSCAN(eps=eps, min_samples=3, algorithm='ball_tree', metric='haversine')
    labels = db.fit_predict(coords_rad)

    group = group.copy()
    group['cluster'] = labels
    group = group[group['cluster'] != -1]  # drop noise

    # Fire ID is now unique per date + cluster
    group['fire_id'] = group['acq_date'].astype(str) + '_C' + group['cluster'].astype(str)
    results.append(group)

# Combine all daily clustered results
clustered_data = pd.concat(results, ignore_index=True)

# Summary per fire_id
summary = (
    clustered_data
    .groupby('fire_id')
    .agg(
        fire_count=('fire_id', 'count'),
        mean_confidence=('confidence', 'mean'),
        state_name=('state_name', 'first'),
        county_name=('county_name', 'first'),
        place_name=('place_name', 'first'),
        acq_date=('acq_date', 'first')
    )
    .reset_index()
)

summary['cluster_confidence_binned']=pd.cut(summary['mean_confidence'],bins=[-1,30,80,101],labels=['Low','Nominal','High'])
summary['fire_count_binned']=pd.cut(summary['fire_count'],bins=[-1,15,50,1000],labels=['Small','Medium','Large'])

output=os.path.join(cwd, "fire_data_enriched.csv")
summary.to_csv(output)
print(f"Saved data to {output}.")

tl_2024_us_state.zip already downloaded.
tl_2024_us_state.shp already extracted.
tl_2024_us_county.zip already downloaded.
tl_2024_us_county.shp already extracted.
tl_2024_01_place.zip already downloaded.
tl_2024_01_place.shp already extracted.
tl_2024_02_place.zip already downloaded.
tl_2024_02_place.shp already extracted.
tl_2024_04_place.zip already downloaded.
tl_2024_04_place.shp already extracted.
tl_2024_05_place.zip already downloaded.
tl_2024_05_place.shp already extracted.
tl_2024_06_place.zip already downloaded.
tl_2024_06_place.shp already extracted.
tl_2024_08_place.zip already downloaded.
tl_2024_08_place.shp already extracted.
tl_2024_09_place.zip already downloaded.
tl_2024_09_place.shp already extracted.
tl_2024_10_place.zip already downloaded.
tl_2024_10_place.shp already extracted.
tl_2024_11_place.zip already downloaded.
tl_2024_11_place.shp already extracted.
tl_2024_12_place.zip already downloaded.
tl_2024_12_place.shp already extracted.
tl_2024_13_place.zip alrea

C:\Users\Josiah\miniconda3\envs\BigProject_1\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www2.census.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Downloaded tl_2024_60_place.zip
Extracting tl_2024_60_place.zip ...
Extracted shapefiles from tl_2024_60_place.zip


C:\Users\Josiah\miniconda3\envs\BigProject_1\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www2.census.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Downloaded tl_2024_66_place.zip
Extracting tl_2024_66_place.zip ...
Extracted shapefiles from tl_2024_66_place.zip


...

Downloaded tl_2024_69_place.zip
Extracting tl_2024_69_place.zip ...
Extracted shapefiles from tl_2024_69_place.zip


C:\Users\Josiah\miniconda3\envs\BigProject_1\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www2.census.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Downloaded tl_2024_72_place.zip
Extracting tl_2024_72_place.zip ...
Extracted shapefiles from tl_2024_72_place.zip


C:\Users\Josiah\miniconda3\envs\BigProject_1\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www2.census.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Downloaded tl_2024_78_place.zip
Extracting tl_2024_78_place.zip ...
Extracted shapefiles from tl_2024_78_place.zip
Loading fire data...
Loading states...
Loading counties...
Loading places...
Mapping fire points to states...
Mapping fire points to counties...
Mapping fire points to places...
Done.


In [7]:
import os
import pandas as pd

path = os.getcwd() + "\\fire_data_enriched.csv"

data=pd.read_csv(path)
data

,Unnamed: 0,fire_id,fire_count,mean_confidence,state_name,county_name,place_name,acq_date,cluster_confidence_binned,fire_count_binned
0,0,2000-11-01_C0,4,72.250000,Virginia,Madison,NaN,2000-11-01,Nominal,Small
1,1,2000-11-01_C1,5,56.200000,Kentucky,Whitley,NaN,2000-11-01,Nominal,Small
2,2,2000-11-01_C2,3,76.666667,Kentucky,Whitley,NaN,2000-11-01,Nominal,Small
3,3,2000-11-01_C3,3,55.333333,Georgia,Gordon,NaN,2000-11-01,Nominal,Small
4,4,2000-11-01_C4,3,89.000000,Kentucky,Whitley,NaN,2000-11-01,High,Small
...,...,...,...,...,...,...,...,...,...,...
126120,126120,2025-01-25_C0,4,88.750000,Hawaii,Hawaii,NaN,2025-01-25,High,Small
126121,126121,2025-01-25_C1,3,98.000000,Hawaii,Hawaii,NaN,2025-01-25,High,Small
126122,126122,2025-01-28_C0,3,92.666667,Hawaii,Hawaii,NaN,2025-01-28,High,Small
126123,126123,2025-01-28_C1,4,71.500000,Kansas,Johnson,Gardner,2025-01-28,Nominal,Small


In [6]:
data['state_name'].sort_values().unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming', nan], dtype=object)

In [189]:
def get_risk_factor(data, state, county=None, place=None, months=None, day=None):
    month_map={'january':1,'february':2,'march':3,'april':4,'may':5,'june':6,'july':7,'august':8,'september':9,'october':10,'november':11,'december':12}
    if months is not None:
        for i in range(len(months)):
            if months[i].lower() in month_map:
                months[i] = month_map[months[i].lower()]

    filtered = data[data['state_name'] == state].drop_duplicates(subset=['acq_date', 'fire_count_binned', 'confidence_binned'])
    filtered['acq_date']=pd.to_datetime(filtered['acq_date'])
    filtered['year'] = filtered['acq_date'].dt.year
    filtered['month'] = filtered['acq_date'].dt.month
    filtered = filtered[filtered['year'] != filtered['year'].max()]
    filtered = filtered[filtered['year'] != filtered['year'].min()]
    years=(filtered['year'].max()-filtered['year'].min())+1
    total=years

    if county is not None:
        filtered = filtered[filtered['county_name'] == county]
    if place is not None:
        filtered = filtered[filtered['place_name'] == place]

    if months is not None:
        filtered = filtered[filtered['month'].isin(months)]
        if len(months) == 1 and day is not None:
            filtered = filtered[filtered['acq_date'].dt.day == day]
        else:
            filtered = filtered.drop_duplicates(subset=['month', 'year', 'fire_count_binned', 'confidence_binned'])
            total=len(months)*years

    else:
        filtered = filtered.drop_duplicates(subset=['year', 'fire_count_binned', 'confidence_binned'])

    filtered=filtered.groupby(['fire_count_binned', 'confidence_binned']).size().to_frame(name='count').reset_index()
    filtered['frequency']=round((filtered['count']/total)*100,2)
    filtered.drop('count',axis=1,inplace=True)

    crosstab = pd.crosstab(
    index=filtered['confidence_binned'],
    columns=filtered['fire_count_binned'],
    values=filtered['frequency'],
    aggfunc='mean',
    margins=False  # <-- no totals
    )

    # Remove axis names but keep labels visible
    crosstab.index.name = None
    crosstab.columns.name = None

    # Style with dark grey borders and header background
    styled_crosstab = (
        crosstab.style
        .format("{:.2f}%")
        .set_table_styles([
            {'selector': 'td, th', 'props': 'border: 2px solid #333333; text-align: center;'},
            {'selector': 'th', 'props': 'background-color: #555555; color: white;'}
        ])
    )

    return styled_crosstab


In [192]:
test=get_risk_factor(summary, state='Colorado', months=['August'])
test

['Moffat' 'Routt' 'Sedgwick' 'Park' 'Las Animas' 'Garfield' 'Teller'
 'Jefferson' 'La Plata' 'Douglas' 'San Miguel' 'Montrose' 'Larimer'
 'Rio Blanco' 'Jackson' 'Bent' 'Grand' 'Phillips' 'Delta' 'Montezuma'
 'Archuleta' 'Mesa' 'Saguache' 'Baca' 'Ouray' 'Gunnison' 'El Paso' 'Eagle'
 'Pueblo' 'Custer' 'Lake' 'Huerfano' 'Costilla' 'Chaffee' 'Conejos'
 'Adams' 'Pitkin' 'Dolores' 'Weld' 'Boulder' 'Fremont' 'Yuma' 'Hinsdale'
 'Mineral' 'Rio Grande' 'Prowers' 'Summit' 'Elbert' 'Clear Creek'
 'San Juan' 'Alamosa']


C:\Users\Josiah\AppData\Local\Temp\ipykernel_14176\2125224529.py:33: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  filtered=filtered.groupby(['fire_count_binned', 'confidence_binned']).size().to_frame(name='count').reset_index()


,Small,Medium,Large
Low,0.00%,0.00%,0.00%
Nominal,68.18%,9.09%,0.00%
High,50.00%,22.73%,4.55%
